### Générateur des phrase

In [ ]:
import pandas as pd
from pathlib import Path


# A DEFINIR : Paramètres de filtrage
patient_id = "P54"
timepoint = "T2"
modality = "FLAIR"


In [ ]:
# 2. Chargement des données
path_str = f"../../data/13_lesion_info_per_label_csv/{patient_id}_{timepoint}_intensity_info.csv"
csv_path = Path(path_str)
no_timepoint = False

if not csv_path.exists():
    path_str = f"../../data/13_lesion_info_per_label_csv/{patient_id}_intensity_info.csv"
    csv_path = Path(path_str)
    no_timepoint = True
    timepoint = "T1"  # Valeur par défaut si le timepoint n'est pas dans le nom du fichier
if not csv_path.exists():
    raise FileNotFoundError(f"Le fichier CSV pour le patient {patient_id} au timepoint {timepoint} n'a pas été trouvé.")

df = pd.read_csv(csv_path)

# 3. Filtrage des données
if not no_timepoint:
    mask = (df['patient'] == patient_id) & (df['timepoint'] == timepoint) & (df['modality'] == modality)
    filtered_df = df[mask].copy()
else:
    mask = (df['patient'] == patient_id) & (df['modality'] == modality)
    filtered_df = df[mask].copy()

if not filtered_df.empty:
    # Identification de la lésion principale (max volume)
    main_lesion = filtered_df.loc[filtered_df['lesion_volume_mm3'].idxmax()]
    
    # --- DÉTERMINATION DYNAMIQUE DE LA ZONE ---
    # Liste toutes les colonnes de localisation (finissant par _prop)
    prop_cols = [c for c in df.columns if c.endswith('_prop')]
    
    # Trouve le nom de la colonne avec la valeur maximale pour cette lésion
    top_zone_col = main_lesion[prop_cols].idxmax()
    top_zone_val = main_lesion[top_zone_col]
    
    # Nettoyage du nom de la zone pour la phrase (ex: 'Zone_77_prop' -> 'Zone 77')
    top_zone_display = top_zone_col.replace('_prop', '').replace('_', ' ')
    
    # 4. Génération de la phrase
    sentence = (
        f"This scan is a {modality} scan of subject {patient_id} at timepoint {timepoint}. "
        f"It reveals a total of {len(filtered_df)} lesions. "
        f"The most significant lesion (ID {int(main_lesion['lesion_id'])}) has a volume of "
        f"{main_lesion['lesion_volume_mm3']:.1f} mm³ and is primarily located in {top_zone_display} "
        f"({top_zone_val*100:.1f}%)."
    )
    
    print(sentence)
else:
    print("Aucune donnée trouvée.")

This scan is a FLAIR scan of a [gender] subject P54 at timepoint T1. It reveals a total of 5 lesions. The most significant lesion (ID 1) has a volume of 531.0 mm³ and is primarily located in Right-Cerebral-White-Matter (54.8%).
